<a href="https://colab.research.google.com/github/vvthakral/data-science-bootcamp/blob/main/amazon_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement

You are given subset of Amazon Transaction data for year 2019. Recommend a second product to the user based on users input (1st product in cart). The recommendation does not need to involve Machine Learning but simply be based on the count of most commonly purchased products (together) from the purchase history in the dataset.To limit the scope, you can target only San Francisco instead of entire USA.


Learnings:<br>
How to handle huge dataset<br>
How to deal with Textual data<br>
How to build prototype recommender system <br>
How to make recommendations more efficient<br>

In [ ]:
import pandas as pd
import numpy as np

# Reading Data

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving amazon_transaction.csv to amazon_transaction.csv


In [ ]:
df = pd.read_csv("amazon_transaction.csv")
df.head(10)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
5,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001"
6,176562,USB-C Charging Cable,1,11.95,04/29/19 13:03,"381 Wilson St, San Francisco, CA 94016"
7,176563,Bose SoundSport Headphones,1,99.99,04/02/19 07:46,"668 Center St, Seattle, WA 98101"
8,176564,USB-C Charging Cable,1,11.95,04/12/19 10:58,"790 Ridge St, Atlanta, GA 30301"
9,176565,Macbook Pro Laptop,1,1700,04/24/19 10:38,"915 Willow St, San Francisco, CA 94016"


#Data Understanding

In [ ]:
#Check for nulls and remove the rows
df.isnull().sum()

Order ID            545
Product             545
Quantity Ordered    545
Price Each          545
Order Date          545
Purchase Address    545
dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
#Get basic stats
df.describe()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
count,186305,186305,186305,186305,186305,186305
unique,178438,20,10,24,142396,140788
top,Order ID,USB-C Charging Cable,1,11.95,Order Date,Purchase Address
freq,355,21903,168552,21903,355,355


In [ ]:
df = df[df['Purchase Address'] != 'Purchase Address']

In [ ]:
df['Order ID'].count() - len(df['Order ID'].unique())

7513

We have 7513 duplicate order id i.e purchase of more than 1 product was made on these order id. 

This is the data we will be using for making recommendations.

In [ ]:
df = df[df['Order ID'].duplicated(keep=False)]
df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
18,176574,Google Phone,1,600,04/03/19 19:42,"20 Hill St, Los Angeles, CA 90001"
19,176574,USB-C Charging Cable,1,11.95,04/03/19 19:42,"20 Hill St, Los Angeles, CA 90001"
30,176585,Bose SoundSport Headphones,1,99.99,04/07/19 11:31,"823 Highland St, Boston, MA 02215"
...,...,...,...,...,...,...
186792,259303,AA Batteries (4-pack),1,3.84,09/20/19 20:18,"106 7th St, Atlanta, GA 30301"
186803,259314,Wired Headphones,1,11.99,09/16/19 00:25,"241 Highland St, Atlanta, GA 30301"
186804,259314,AAA Batteries (4-pack),2,2.99,09/16/19 00:25,"241 Highland St, Atlanta, GA 30301"
186841,259350,Google Phone,1,600,09/30/19 13:49,"519 Maple St, San Francisco, CA 94016"


#Feature Engineering

In [ ]:
#extract city from address
def get_city(address):
  return address.split(",")[1].strip(" ")

df['city'] = df['Purchase Address'].apply(get_city)
df.head(10)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,city
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",Los Angeles
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",Los Angeles
18,176574,Google Phone,1,600,04/03/19 19:42,"20 Hill St, Los Angeles, CA 90001",Los Angeles
19,176574,USB-C Charging Cable,1,11.95,04/03/19 19:42,"20 Hill St, Los Angeles, CA 90001",Los Angeles
30,176585,Bose SoundSport Headphones,1,99.99,04/07/19 11:31,"823 Highland St, Boston, MA 02215",Boston
31,176585,Bose SoundSport Headphones,1,99.99,04/07/19 11:31,"823 Highland St, Boston, MA 02215",Boston
32,176586,AAA Batteries (4-pack),2,2.99,04/10/19 17:00,"365 Center St, San Francisco, CA 94016",San Francisco
33,176586,Google Phone,1,600,04/10/19 17:00,"365 Center St, San Francisco, CA 94016",San Francisco
119,176672,Lightning Charging Cable,1,14.95,04/12/19 11:07,"778 Maple St, New York City, NY 10001",New York City
120,176672,USB-C Charging Cable,1,11.95,04/12/19 11:07,"778 Maple St, New York City, NY 10001",New York City


In [ ]:
#select data for San Francisco only
df = df[df['city']=='San Francisco']
print(f"Number of orders with more than 1 product purchase in San Francisco are {df['city'].count()}")

Number of orders with more than 1 product purchase in San Francisco are 3562


In [ ]:
#Use the apply function to merge products in same order id.
#refer session notebook or the below link
#https://stackoverflow.com/questions/27298178/concatenate-strings-from-several-rows-using-pandas-groupby

In [ ]:
#Task 1
#Merge products with same order id
'''
After merger, products in same order id should be merged into 1 string
eg: Google Phone, Wired Headphones
'''

df = df.groupby(['Order ID'])['Product'].apply(lambda x: ','.join(x)).reset_index()
df.head()

,Order ID,Product
0,141450,"Google Phone,Bose SoundSport Headphones"
1,141782,"27in FHD Monitor,Bose SoundSport Headphones"
2,141795,"iPhone,Wired Headphones"
3,141843,"AA Batteries (4-pack),AAA Batteries (4-pack)"
4,141946,"USB-C Charging Cable,Wired Headphones"


#Recommendation

In [ ]:
#Task 2
#Make recommendation
'''
Make simple recommendation, just based on count of combinations from previous purchase history.
eg: 
google phone, wired headphones 5 times
google phone, usb type c 10 times
so recommend usb type c cable if customer has google phone in their cart
'''
combinations = {}
for _ in df['Product']:
  p = _.split(',')
  p.sort()
  for i in range(len(p)):
    for j in range(i+1,len(p)):
      if (p[i],p[j]) in combinations:
        combinations[(p[i],p[j])] +=1
      else:
        combinations[(p[i],p[j])]=1

print(combinations)

{('Bose SoundSport Headphones', 'Google Phone'): 52, ('27in FHD Monitor', 'Bose SoundSport Headphones'): 11, ('Wired Headphones', 'iPhone'): 119, ('AA Batteries (4-pack)', 'AAA Batteries (4-pack)'): 17, ('USB-C Charging Cable', 'Wired Headphones'): 60, ('Bose SoundSport Headphones', 'Lightning Charging Cable'): 22, ('Google Phone', 'iPhone'): 2, ('20in Monitor', 'Wired Headphones'): 3, ('Apple Airpods Headphones', 'USB-C Charging Cable'): 13, ('Google Phone', 'USB-C Charging Cable'): 245, ('Google Phone', 'Wired Headphones'): 93, ('USB-C Charging Cable', 'Vareebadd Phone'): 91, ('Lightning Charging Cable', 'USB-C Charging Cable'): 18, ('AA Batteries (4-pack)', 'Macbook Pro Laptop'): 5, ('Lightning Charging Cable', 'iPhone'): 254, ('Apple Airpods Headphones', 'Lightning Charging Cable'): 26, ('Apple Airpods Headphones', 'iPhone'): 96, ('Apple Airpods Headphones', 'Bose SoundSport Headphones'): 14, ('Lightning Charging Cable', 'Wired Headphones'): 39, ('34in Ultrawide Monitor', 'Lightnin

In [ ]:
#Make list out of dictionary
comb_list = list(combinations.items())
print(comb_list)

[(('Bose SoundSport Headphones', 'Google Phone'), 52), (('27in FHD Monitor', 'Bose SoundSport Headphones'), 11), (('Wired Headphones', 'iPhone'), 119), (('AA Batteries (4-pack)', 'AAA Batteries (4-pack)'), 17), (('USB-C Charging Cable', 'Wired Headphones'), 60), (('Bose SoundSport Headphones', 'Lightning Charging Cable'), 22), (('Google Phone', 'iPhone'), 2), (('20in Monitor', 'Wired Headphones'), 3), (('Apple Airpods Headphones', 'USB-C Charging Cable'), 13), (('Google Phone', 'USB-C Charging Cable'), 245), (('Google Phone', 'Wired Headphones'), 93), (('USB-C Charging Cable', 'Vareebadd Phone'), 91), (('Lightning Charging Cable', 'USB-C Charging Cable'), 18), (('AA Batteries (4-pack)', 'Macbook Pro Laptop'), 5), (('Lightning Charging Cable', 'iPhone'), 254), (('Apple Airpods Headphones', 'Lightning Charging Cable'), 26), (('Apple Airpods Headphones', 'iPhone'), 96), (('Apple Airpods Headphones', 'Bose SoundSport Headphones'), 14), (('Lightning Charging Cable', 'Wired Headphones'), 39)

In [ ]:
#sort list based on the number of occurences/combination count
comb_list = sorted(comb_list,key=lambda x:x[1],reverse=True)
print(comb_list)

[(('Lightning Charging Cable', 'iPhone'), 254), (('Google Phone', 'USB-C Charging Cable'), 245), (('Wired Headphones', 'iPhone'), 119), (('Apple Airpods Headphones', 'iPhone'), 96), (('Google Phone', 'Wired Headphones'), 93), (('USB-C Charging Cable', 'Vareebadd Phone'), 91), (('USB-C Charging Cable', 'Wired Headphones'), 60), (('Bose SoundSport Headphones', 'Google Phone'), 52), (('Vareebadd Phone', 'Wired Headphones'), 44), (('Lightning Charging Cable', 'Wired Headphones'), 39), (('Apple Airpods Headphones', 'Lightning Charging Cable'), 26), (('AA Batteries (4-pack)', 'Wired Headphones'), 26), (('AA Batteries (4-pack)', 'Lightning Charging Cable'), 26), (('Apple Airpods Headphones', 'Wired Headphones'), 25), (('AA Batteries (4-pack)', 'USB-C Charging Cable'), 25), (('Bose SoundSport Headphones', 'Lightning Charging Cable'), 22), (('Bose SoundSport Headphones', 'USB-C Charging Cable'), 22), (('AAA Batteries (4-pack)', 'USB-C Charging Cable'), 21), (('Bose SoundSport Headphones', 'Vare

In [ ]:
def recommender(product):
  '''
  code for recommendation
  Simple method: return any product that was purchased with the product in cart
  better approach: return product with highest match count
  '''
  global comb_list
  product = product.lower().strip() # clean,format the user input
  for i in comb_list:
    if i[0][0].lower() == product:
      return i[0][1]
    elif i[0][1].lower() == product:
      return i[0][0]
  return 'We are still upgrading our recommendation system!'

product = 'google phone'
print(recommender(product))

USB-C Charging Cable
